### 📦 Instalación de librerías para datos climáticos y espaciales

Instala paquetes necesarios para trabajar con datos NetCDF y raster: `xarray`, `rioxarray` y `netCDF4`.


In [1]:
pip install xarray rioxarray netcdf4 netCDF4


Note: you may need to restart the kernel to use updated packages.


### 🌍 Descarga de datos ERA5-Land desde Copernicus con `cdsapi`

Este script automatiza la descarga de variables meteorológicas del conjunto de reanálisis **ERA5-Land** para el departamento de Caldas, Colombia.

- Se utiliza la API oficial de **Copernicus Climate Data Store (CDS)** mediante `cdsapi`.
- Se descarga por quincena (días 1-15 y 16-fin de mes) para los años especificados.
- El área geográfica se define con coordenadas: `[Norte, Oeste, Sur, Este]`.
- Las variables incluyen temperatura, radiación, viento, presión y precipitación.
- Los archivos `.grib` descargados se guardan y, si están comprimidos, se descomprimen automáticamente.

📁 **Estructura de carpetas**:
- `descargas_zips`: contiene archivos originales `.grib` o `.zip`.
- `datos_grib_validos`: almacena los archivos organizados por quincena y mes.

⏱️ Se introduce una pausa de 3 segundos entre descargas para evitar sobrecargar el servidor.

> ⚠️ Asegúrate de tener una cuenta en [CDS](https://cds.climate.copernicus.eu/) y configurar tu archivo `.cdsapirc`.



In [ ]:
import cdsapi
import os
import zipfile
from time import sleep

client = cdsapi.Client()

# Parámetros
years = ["2016", "2017", "2018", "2024"]
months = [f"{m:02d}" for m in range(1, 13)]
hours = [f"{h:02d}:00" for h in range(24)]
area = [5.4, -75.75, 4.8, -75.35]  # Caldas
variables = [
    "2m_dewpoint_temperature", "2m_temperature", "surface_thermal_radiation_downwards",
    "surface_solar_radiation_downwards", "10m_u_component_of_wind",
    "10m_v_component_of_wind", "surface_pressure", "total_precipitation"
]

# Carpetas
carpeta_zips = "descargas_zips"
carpeta_gribs = "datos_grib_validos"
os.makedirs(carpeta_zips, exist_ok=True)
os.makedirs(carpeta_gribs, exist_ok=True)

def dias_en_mes(año, mes):
    mes = int(mes)
    if mes == 2:
        return 29 if int(año) % 4 == 0 else 28
    return 30 if mes in [4, 6, 9, 11] else 31

# Descarga y extracción
for year in years:
    for month in months:
        dias = dias_en_mes(year, month)

        for parte, (d1, d2) in enumerate([(1, 15), (16, dias)], start=1):
            dias_rango = [f"{d:02d}" for d in range(d1, d2 + 1)]

            nombre_archivo = f"era5land_{year}_{month}_q{parte}"
            zip_path = os.path.join(carpeta_zips, f"{nombre_archivo}.grib")
            output_dir = os.path.join(carpeta_gribs, nombre_archivo)
            os.makedirs(output_dir, exist_ok=True)

            if os.path.exists(zip_path):
                print(f"✅ Ya existe: {zip_path}, saltando descarga...")
            else:
                print(f"📦 Descargando {nombre_archivo}...")
                try:
                    client.retrieve(
                        "reanalysis-era5-land",
                        {
                            "product_type": "reanalysis",
                            "variable": variables,
                            "year": year,
                            "month": month,
                            "day": dias_rango,
                            "time": hours,
                            "format": "grib",
                            "area": area,
                            "grid": [0.1, 0.1]
                        }
                    ).download(zip_path)
                    print(f"✅ Descargado: {zip_path}")
                except Exception as e:
                    print(f"❌ Error en {nombre_archivo}: {e}")
                    continue

            # Verificar si es ZIP
            with open(zip_path, "rb") as f:
                firma = f.read(4)

            if firma == b'PK\x03\x04':
                print(f"📂 {zip_path} es un ZIP, descomprimiendo...")
                try:
                    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                        zip_ref.extractall(output_dir)
                    print(f"✅ Descomprimido en: {output_dir}\n")
                except Exception as e:
                    print(f"❌ Error al descomprimir {zip_path}: {e}")
            else:
                print(f"⚠️ {zip_path} no es ZIP, moviendo a carpeta final...")
                # Mover a carpeta destino
                nuevo_path = os.path.join(output_dir, f"{nombre_archivo}.grib")
                os.rename(zip_path, nuevo_path)
                print(f"✅ Movido a: {nuevo_path}\n")

            sleep(3)  # Para evitar saturar el servidor


### 📂 Exploración de variables contenidas en archivos `.grib`

Este bloque recorre una carpeta raíz (`datos_grib_validos`) que contiene subcarpetas con archivos `data.grib` descomprimidos. Su objetivo es identificar qué variables meteorológicas contiene cada archivo:

- Utiliza `xarray` con el motor `cfgrib` para leer archivos `.grib`, formato común en datos meteorológicos.
- Por cada subcarpeta:
  - Abre el archivo `data.grib`.
  - Extrae las variables disponibles (`ds.data_vars`).
  - Las guarda en un diccionario `variables_por_archivo`, donde la llave es el nombre del subdirectorio.
  - Muestra en consola las variables leídas o un mensaje de error si el archivo no existe o no puede abrirse.

✅ Esto es útil para auditar la estructura de los datos descargados y confirmar que las variables esperadas están presentes en cada archivo quincenal o mensual.


In [3]:
import os
import xarray as xr

# Ruta raíz donde están los .grib descomprimidos
carpeta_gribs = "datos_grib_validos"

# Diccionario para guardar el conjunto de variables por archivo
variables_por_archivo = {}

# Leer todos los archivos y guardar sus variables
for subdir in sorted(os.listdir(carpeta_gribs)):
    ruta = os.path.join(carpeta_gribs, subdir, "data.grib")
    
    if os.path.exists(ruta):
        try:
            ds = xr.open_dataset(ruta, engine="cfgrib")
            variables = set(ds.data_vars)
            variables_por_archivo[subdir] = variables
            print(f"✅ {subdir}: {sorted(variables)}")
        except Exception as e:
            print(f"❌ Error al leer {subdir}: {e}")
    else:
        print(f"⚠️ Archivo no encontrado: {ruta}")


Ignoring index file 'datos_grib_validos\\era5land_2016_01_q1\\data.grib.5b7b6.idx' older than GRIB file
c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
Ignoring index file 'datos_grib_validos\\era5land_2016_01_q2\\data.grib.5b7b6.idx' older than GRIB file


✅ era5land_2016_01_q1: ['d2m', 'sp', 'ssrd', 'strd', 't2m', 'tp', 'u10', 'v10']


c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
Ignoring index file 'datos_grib_validos\\era5land_2016_02_q1\\data.grib.5b7b6.idx' older than GRIB file


✅ era5land_2016_01_q2: ['d2m', 'sp', 'ssrd', 'strd', 't2m', 'tp', 'u10', 'v10']


c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
Ignoring index file 'datos_grib_validos\\era5land_2016_02_q2\\data.grib.5b7b6.idx' older than GRIB file


...

c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default

✅ era5land_2024_07_q2: ['d2m', 'sp', 'ssrd', 'strd', 't2m', 'tp', 'u10', 'v10']
✅ era5land_2024_08_q1: ['d2m', 'sp', 'ssrd', 'strd', 't2m', 'tp', 'u10', 'v10']
✅ era5land_2024_08_q2: ['d2m', 'sp', 'ssrd', 'strd', 't2m', 'tp', 'u10', 'v10']
✅ era5land_2024_09_q1: ['d2m', 'sp', 'ssrd', 'strd', 't2m', 'tp', 'u10', 'v10']


c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default

✅ era5land_2024_09_q2: ['d2m', 'sp', 'ssrd', 'strd', 't2m', 'tp', 'u10', 'v10']
✅ era5land_2024_10_q1: ['d2m', 'sp', 'ssrd', 'strd', 't2m', 'tp', 'u10', 'v10']
✅ era5land_2024_10_q2: ['d2m', 'sp', 'ssrd', 'strd', 't2m', 'tp', 'u10', 'v10']
✅ era5land_2024_11_q1: ['d2m', 'sp', 'ssrd', 'strd', 't2m', 'tp', 'u10', 'v10']
✅ era5land_2024_11_q2: ['d2m', 'sp', 'ssrd', 'strd', 't2m', 'tp', 'u10', 'v10']
✅ era5land_2024_12_q1: ['d2m', 'sp', 'ssrd', 'strd', 't2m', 'tp', 'u10', 'v10']
✅ era5land_2024_12_q2: ['d2m', 'sp', 'ssrd', 'strd', 't2m', 'tp', 'u10', 'v10']


c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default

### ✅ Verificación de consistencia en las variables entre archivos `.grib`

Este fragmento compara las variables meteorológicas contenidas en todos los archivos `.grib` respecto al primer archivo leído:

- Se toma el primer conjunto de variables como **referencia base**.
- Se comparan todos los demás archivos contra esa base.
- Si un archivo tiene variables diferentes, se guarda en el diccionario `inconsistencias` con las diferencias identificadas mediante `symmetric_difference`.
- Finalmente:
  - Si no hay inconsistencias, imprime un mensaje de éxito.
  - Si hay diferencias, las lista archivo por archivo indicando qué variables faltan o sobran.

📌 Esto permite asegurar que todos los archivos `.grib` tienen las mismas variables antes de integrarlos o analizarlos en conjunto.


In [4]:
# Comparar todas las variables con respecto al primer archivo
base_archivo, base_vars = next(iter(variables_por_archivo.items()))
inconsistencias = {}

for nombre, vars_actuales in variables_por_archivo.items():
    if vars_actuales != base_vars:
        inconsistencias[nombre] = vars_actuales.symmetric_difference(base_vars)

# Mostrar resultados
if not inconsistencias:
    print("🎯 Todos los archivos contienen las mismas variables.")
else:
    print("⚠️ Archivos con diferencias en variables:")
    for nombre, dif in inconsistencias.items():
        print(f"- {nombre}: diferencia = {sorted(dif)}")


🎯 Todos los archivos contienen las mismas variables.


### 🧩 Lectura y concatenación de múltiples archivos `.grib` en un solo Dataset

Este bloque tiene como objetivo unificar todos los archivos `data.grib` ubicados en subdirectorios dentro de la carpeta `datos_grib_validos`:

1. 🔄 **Recorrido ordenado**:
   - Se recorre la lista de subdirectorios en orden alfabético (para mantener el orden temporal).
   - Cada archivo `data.grib` se abre con `xarray` usando el motor `cfgrib`.

2. 📦 **Carga y almacenamiento**:
   - Los datasets leídos se agregan a una lista llamada `datasets`.

3. 🔗 **Concatenación**:
   - Todos los datasets son concatenados a lo largo de la dimensión temporal (`dim="time"`) usando `xr.concat`.

4. 📋 **Resumen del resultado**:
   - Finalmente, se imprime el resumen del dataset resultante (`ds_concatenado`), que ahora contiene datos meteorológicos de múltiples fechas en una sola estructura.

✅ Este paso es clave para preparar los datos en un solo objeto que pueda ser analizado o transformado más fácilmente.


In [5]:
import os
import xarray as xr

# Carpeta donde están los subdirectorios con los GRIB descomprimidos
carpeta = "datos_grib_validos"

# Lista para guardar cada Dataset
datasets = []

# Recorremos en orden alfabético para mantener orden temporal
for subdir in sorted(os.listdir(carpeta)):
    path = os.path.join(carpeta, subdir, "data.grib")
    if os.path.exists(path):
        try:
            ds = xr.open_dataset(path, engine="cfgrib")
            datasets.append(ds)
            print(f"✅ Añadido: {subdir}")
        except Exception as e:
            print(f"❌ Error en {subdir}: {e}")

# Concatenar por dimensión 'time'
ds_concatenado = xr.concat(datasets, dim="time")

# Mostrar resumen
print(ds_concatenado)


Ignoring index file 'datos_grib_validos\\era5land_2016_01_q1\\data.grib.5b7b6.idx' older than GRIB file
c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
Ignoring index file 'datos_grib_validos\\era5land_2016_01_q2\\data.grib.5b7b6.idx' older than GRIB file


✅ Añadido: era5land_2016_01_q1


c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
Ignoring index file 'datos_grib_validos\\era5land_2016_02_q1\\data.grib.5b7b6.idx' older than GRIB file


✅ Añadido: era5land_2016_01_q2


c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
Ignoring index file 'datos_grib_validos\\era5land_2016_02_q2\\data.grib.5b7b6.idx' older than GRIB file


...

✅ Añadido: era5land_2024_08_q1
✅ Añadido: era5land_2024_08_q2
✅ Añadido: era5land_2024_09_q1


c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default

✅ Añadido: era5land_2024_09_q2
✅ Añadido: era5land_2024_10_q1
✅ Añadido: era5land_2024_10_q2


c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
c:\Users\jessi\AppData\Local\Programs\Python\Python311\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default

✅ Añadido: era5land_2024_11_q1
✅ Añadido: era5land_2024_11_q2
✅ Añadido: era5land_2024_12_q1
✅ Añadido: era5land_2024_12_q2
<xarray.Dataset> Size: 42MB
Dimensions:     (time: 1558, step: 24, latitude: 7, longitude: 5)
Coordinates:
    number      int64 8B 0
  * time        (time) datetime64[ns] 12kB 2015-12-31 2016-01-01 ... 2024-12-31
  * step        (step) timedelta64[ns] 192B 01:00:00 ... 1 days 00:00:00
    surface     float64 8B 0.0
  * latitude    (latitude) float64 56B 5.4 5.3 5.2 5.1 5.0 4.9 4.8
  * longitude   (longitude) float64 40B -75.75 -75.65 -75.55 -75.45 -75.35
    valid_time  (time, step) datetime64[ns] 299kB 2015-12-31T01:00:00 ... 202...
Data variables:
    d2m         (time, step, latitude, longitude) float32 5MB nan nan ... nan
    t2m         (time, step, latitude, longitude) float32 5MB nan nan ... nan
    strd        (time, step, latitude, longitude) float32 5MB nan nan ... nan
    ssrd        (time, step, latitude, longitude) float32 5MB nan nan ... nan
    u10

### 🔄 Conversión del `xarray.Dataset` a `pandas.DataFrame` en formato largo

En esta etapa se transforma el `Dataset` concatenado (`ds_concatenado`) en un `DataFrame` plano para facilitar análisis y visualizaciones:

- Se usa `.to_dataframe()` para convertir el objeto multidimensional de `xarray` a un `DataFrame` de `pandas`.
- `.reset_index()` elimina los índices jerárquicos (como tiempo, latitud y longitud), convirtiéndolos en columnas explícitas.
- El resultado, `datos_meteorologicos`, tiene una estructura **larga**: cada fila representa una observación única en un punto geográfico y momento específico, con las variables meteorológicas como columnas.

📌 Este formato es ideal para análisis con `pandas`, creación de gráficos, limpieza de datos o exportación a `.csv`.


In [ ]:
# Convertir el dataset a DataFrame (formato largo)
datos_meteorologicos = ds_concatenado.to_dataframe().reset_index()
print(datos_meteorologicos)


### 🧹 Limpieza del DataFrame meteorológico: filtrado temporal y eliminación de columnas innecesarias

Este bloque realiza una limpieza del DataFrame `datos_meteorologicos` para dejarlo con una estructura más clara y coherente:

1. 🕒 **Conversión de la columna `valid_time` a tipo datetime**:
   - Se asegura que los valores sean reconocidos como fechas para poder aplicar filtros temporales.

2. 📆 **Filtrado de registros**:
   - Se eliminan todas las filas correspondientes al año 2015, dejando solo datos de 2016 en adelante.

3. 🧼 **Eliminación de columnas innecesarias**:
   - Se eliminan las columnas:
     - `time`: ya no es necesaria, pues se conserva solo `valid_time` como referencia temporal.
     - `step`: información redundante sobre el paso de tiempo (repetida en `valid_time`).
     - `surface` y `number`: metadatos internos del archivo `.grib` que no son útiles para el análisis.

✅ Después de esta limpieza, el DataFrame queda más compacto y enfocado únicamente en las variables meteorológicas relevantes por hora y ubicación geográfica.


In [ ]:
import pandas as pd
# Convertir a datetime si es necesario
datos_meteorologicos['valid_time'] = pd.to_datetime(datos_meteorologicos['valid_time'])

# Eliminar registros del año 2015
datos_meteorologicos = datos_meteorologicos[datos_meteorologicos['valid_time'].dt.year > 2015]

# Asegúrate de que 'valid_time' es tipo datetime
datos_meteorologicos['valid_time'] = pd.to_datetime(datos_meteorologicos['valid_time'])

# Filtrar registros para conservar solo los posteriores a 2015
datos_meteorologicos = datos_meteorologicos[datos_meteorologicos['valid_time'].dt.year > 2015]

# Eliminar las columnas 'time' y 'step'
datos_meteorologicos = datos_meteorologicos.drop(columns=['time', 'step'])

# Eliminar las columnas 'suface' y 'number'
datos_meteorologicos = datos_meteorologicos.drop(columns=['surface', 'number'])


### 🔍 Verificación de valores faltantes en variables clave

Este bloque revisa si existen valores nulos (`NaN`) en las principales variables meteorológicas del DataFrame:

- Se define la lista `variables_necesarias` con las variables clave a analizar:
  - `t2m`: temperatura a 2 metros
  - `d2m`: temperatura del punto de rocío
  - `ssrd`: radiación solar descendente en superficie
  - `strd`: radiación térmica de onda larga en superficie
  - `tp`: precipitación total
  - `u10`, `v10`: componentes zonal y meridional del viento a 10 metros

- Se utiliza `.isna().sum()` para contar la cantidad de valores faltantes por variable.

📋 El resultado se imprime como tabla, permitiendo evaluar la calidad de los datos y decidir si se necesita imputación, interpolación o eliminación de registros.


In [ ]:
import numpy as np
import pandas as pd

# Verificar si hay NaN en las columnas necesarias
variables_necesarias = ['t2m', 'd2m', 'ssrd', 'strd', 'tp', 'u10', 'v10']
na_check = datos_meteorologicos[variables_necesarias].isna().sum()

print("Valores faltantes por variable:")
print(na_check)

# Si deseas eliminar las filas con NaN antes del cálculo:
datos_meteorologicos.dropna(subset=variables_necesarias, inplace=True)


Valores faltantes por variable:
t2m     79835
d2m     79835
ssrd    79835
strd    79835
tp      79835
u10     79835
v10     79835
dtype: int64


### 🧮 Cálculo de variables meteorológicas derivadas

Este bloque transforma las variables originales (en unidades físicas estandarizadas) en medidas más interpretables y útiles para el análisis climático:

| **Variable requerida** | **Significado**                    | **Fórmula / Cálculo**                                                                                                                                                      | **Variables base necesarias (ERA5)**                 | **Unidades originales**     | **Nombre en DataFrame**      |
|------------------------|------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------|-----------------------------|------------------------------|
| `TA_1_1_1`             | Temperatura del aire a 2 m (°C)    | `TA_1_1_1 = t2m - 273.15`                                                                                                                                                   | `2m_temperature`                                     | Kelvin (K)                  | `t2m`                        |
| `RH_1_1_1`             | Humedad relativa (%)               | `e = 6.112 × exp[(17.67 × d2m)/(d2m + 243.5)]`<br>`e_s = 6.112 × exp[(17.67 × t2m)/(t2m + 243.5)]`<br>`RH = 100 × (e / e_s)`                                                 | `2m_temperature`, `2m_dewpoint_temperature`          | Kelvin (K)                  | `t2m`, `d2m`                 |
| `VPD_PI`               | Déficit de presión de vapor (hPa)  | `VPD = e_s - e` usando las mismas fórmulas anteriores                                                                                                                      | `2m_temperature`, `2m_dewpoint_temperature`          | Kelvin (K)                  | `t2m`, `d2m`                 |
| `SW_IN_1_1_1`          | Radiación solar incidente (W/m²)   | `SW_IN_1_1_1 = ssrd / 3600`                                                                                                                                                | `surface_solar_radiation_downwards`                  | Joules por m² (J/m²)        | `ssrd`                       |
| `LW_IN`                | Radiación térmica incidente (W/m²) | `LW_IN = strd / 3600`                                                                                                                                                      | `surface_thermal_radiation_downwards`                | Joules por m² (J/m²)        | `strd`                       |
| `P`                    | Precipitación total (mm)           | `P = tp * 1000`                                                                                                                                                            | `total_precipitation`                                | Metros (m)                  | `tp`                         |
| `WS`                   | Velocidad del viento a 10 m (m/s)  | `WS = √(u10² + v10²)`                                                                                                                                                      | `10m_u_component_of_wind`, `10m_v_component_of_wind` | Metros por segundo (m/s)    | `u10`, `v10`                 |


In [28]:
# 1. Temperatura del aire a 2m (°C)
datos_meteorologicos["TA_1_1_1"] = datos_meteorologicos["t2m"] - 273.15

# 2. Humedad relativa (%)
Td = datos_meteorologicos["d2m"] - 273.15
T = datos_meteorologicos["t2m"] - 273.15

e = 6.112 * np.exp((17.67 * Td) / (Td + 243.5))
es = 6.112 * np.exp((17.67 * T) / (T + 243.5))
datos_meteorologicos["RH_1_1_1"] = 100 * (e / es)

# 3. Déficit de presión de vapor (hPa)
datos_meteorologicos["VPD_PI"] = es - e

# 4. Radiación solar incidente (W/m²)
datos_meteorologicos["SW_IN_1_1_1"] = datos_meteorologicos["ssrd"] / 3600

# 5. Radiación térmica incidente (W/m²)
datos_meteorologicos["LW_IN"] = datos_meteorologicos["strd"] / 3600

# 6. Precipitación total (mm)
datos_meteorologicos["P"] = datos_meteorologicos["tp"] * 1000

# 7. Velocidad del viento a 10m (m/s)
datos_meteorologicos["WS"] = np.sqrt(datos_meteorologicos["u10"]**2 + datos_meteorologicos["v10"]**2)

# Mostrar las primeras filas para revisar
print("\nVariables calculadas:")
print(datos_meteorologicos[["TA_1_1_1", "RH_1_1_1", "VPD_PI", "SW_IN_1_1_1", "LW_IN", "P", "WS"]].head())



Variables calculadas:
      TA_1_1_1   RH_1_1_1    VPD_PI  SW_IN_1_1_1        LW_IN         P  \
805  18.632233  89.168648  2.324497  3950.961670  8657.380859  1.754855   
806  22.028717  83.891258  4.264734  4126.850098  8441.751953  1.020242   
807  22.978912  77.646881  6.269915  4308.090332  8250.395508  0.706960   
808  18.498444  74.938644  5.333482  4491.039551  8070.979492  0.584409   
809  14.639069  71.822021  4.691209  4676.913574  7995.352051  0.522897   

           WS  
805  0.685388  
806  0.547065  
807  0.139769  
808  0.419850  
809  0.774673  


### 🧾 Selección de variables finales para el análisis

En este paso se crea un nuevo DataFrame que conserva únicamente las columnas relevantes para el análisis climático y espacial:

- `valid_time`: fecha y hora de la observación.
- `latitude` y `longitude`: coordenadas geográficas del punto de medición.
- `TA_1_1_1`: temperatura del aire a 2 metros (°C).
- `RH_1_1_1`: humedad relativa (%).
- `VPD_PI`: déficit de presión de vapor (hPa).
- `SW_IN_1_1_1`: radiación solar incidente (W/m²).
- `LW_IN`: radiación térmica incidente (W/m²).
- `P`: precipitación acumulada (mm).
- `WS`: velocidad del viento (m/s).

🔎 Este filtrado deja el DataFrame listo para análisis exploratorios, visualizaciones o modelos predictivos, con una estructura compacta y solo las variables necesarias.


In [29]:
# Paso 6: Dejar solo columnas finales deseadas
columnas_finales = [
    "valid_time", "latitude", "longitude",
    "TA_1_1_1", "RH_1_1_1", "VPD_PI",
    "SW_IN_1_1_1", "LW_IN", "P", "WS"
]

datos_meteorologicos = datos_meteorologicos[columnas_finales]

### ⏱️ Interpolación temporal a intervalos de 30 minutos y enriquecimiento de variables temporales

Este bloque duplica la frecuencia temporal de los datos meteorológicos, creando observaciones intermedias cada 30 minutos, y añade columnas útiles para análisis horarios y estacionales:

---

#### 🔁 Interpolación por duplicación temporal
- Se crea una copia del DataFrame y se le suma 30 minutos a la columna `valid_time`.
- Luego, se combinan los datos originales y los desplazados usando `pd.concat`.
- El resultado son registros interpolados a intervalos de **30 minutos**, sin aplicar ninguna transformación en los valores aún (esto es útil si luego se quiere interpolar suavemente o realizar simulaciones más frecuentes).

---

#### 📏 Reordenamiento espacial y temporal
- Los datos se ordenan por `valid_time`, `latitude` y `longitude` para mantener coherencia en el análisis espacio-temporal.

---

#### 📆 Enriquecimiento con variables de tiempo
- `doy`: día del año (1 a 365 o 366), útil para análisis estacionales o ciclos anuales.
- `hour`: hora del día en formato decimal (ej. 13.5 para las 13:30), útil para estudios diarios o ciclos circadianos.

✅ Esta transformación prepara los datos para modelos o visualizaciones que requieren mayor resolución temporal o segmentación por hora/día del año.


In [ ]:
import pandas as pd

# Asegúrate de que 'valid_time' sea de tipo datetime
datos_meteorologicos['valid_time'] = pd.to_datetime(datos_meteorologicos['valid_time'])

# Creamos una copia de las filas originales con valid_time + 30 minutos
datos_interpolados = datos_meteorologicos.copy()
datos_interpolados['valid_time'] = datos_interpolados['valid_time'] + pd.Timedelta(minutes=30)

# Unimos el original con la versión desplazada
datos_meteorologicos = pd.concat([datos_meteorologicos, datos_interpolados], ignore_index=True)

# Ordenamos por tiempo, latitud y longitud para mantener coherencia espacial-temporal
datos_meteorologicos = datos_meteorologicos.sort_values(by=['valid_time', 'latitude', 'longitude']).reset_index(drop=True)

# Asegúrate de que 'valid_time' sea datetime (por si no lo está aún)
datos_meteorologicos['valid_time'] = pd.to_datetime(datos_meteorologicos['valid_time'])

# Día del año (1-365/366)
datos_meteorologicos['doy'] = datos_meteorologicos['valid_time'].dt.dayofyear

# Hora en formato decimal (por ejemplo, 13.5 para 13:30)
datos_meteorologicos['hour'] = (
    datos_meteorologicos['valid_time'].dt.hour +
    datos_meteorologicos['valid_time'].dt.minute / 60
)


In [32]:
print(datos_meteorologicos)

                 valid_time  latitude  longitude   TA_1_1_1   RH_1_1_1  \
0       2016-01-01 00:00:00       4.8     -75.75  19.595123  87.700951   
1       2016-01-01 00:00:00       4.8     -75.65  18.885162  88.978897   
2       2016-01-01 00:00:00       4.8     -75.55  16.120514  88.946922   
3       2016-01-01 00:00:00       4.8     -75.45  11.233795  87.099213   
4       2016-01-01 00:00:00       4.8     -75.35   9.225983  82.682060   
...                     ...       ...        ...        ...        ...   
2456155 2024-12-31 23:30:00       5.4     -75.75  17.574127  83.456444   
2456156 2024-12-31 23:30:00       5.4     -75.65  19.493073  83.802109   
2456157 2024-12-31 23:30:00       5.4     -75.55  19.164948  86.257156   
2456158 2024-12-31 23:30:00       5.4     -75.45  15.528229  88.156494   
2456159 2024-12-31 23:30:00       5.4     -75.35  12.521393  89.229492   

           VPD_PI  SW_IN_1_1_1        LW_IN         P        WS  doy  hour  
0        2.802898  4627.227051  89

In [33]:
# Guardar en CSV
datos_meteorologicos.to_csv("datos_meteorologicos_era5land.csv", index=False)

print("📁 CSV guardado como 'datos_meteorologicos_era5land.csv'")

📁 CSV guardado como 'datos_meteorologicos_era5land.csv'


In [2]:
pip install folium branca ipywidgets openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Mapa Datos meteorológicos
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from branca.colormap import linear
from ipywidgets import interact, Dropdown
import datetime

# 📥 Cargar datos
df = pd.read_excel("datos_meteorologicos_era5land_solo_2018.xlsx", parse_dates=["valid_time"])


# 🧠 Diccionario de nombres legibles
nombre_variables_legibles = {
    "TA_1_1_1": "Temperatura del aire (°C)",
    "RH_1_1_1": "Humedad relativa (%)",
    "VPD_PI": "Déficit de presión de vapor (hPa)",
    "SW_IN_1_1_1": "Radiación solar incidente (W/m²)",
    "LW_IN": "Radiación térmica incidente (W/m²)",
    "P": "Precipitación total (mm)",
    "WS": "Velocidad del viento (m/s)"
}

nombres_variables = list(nombre_variables_legibles.keys())

# 🗓️ Fechas disponibles
fechas_disponibles = sorted(df["valid_time"].dt.date.unique())
fecha_default = pd.to_datetime("2018-12-31").date()

if fecha_default not in fechas_disponibles:
    print("⚠️ La fecha 2024-12-31 no está en los datos, se usará la más reciente.")
    fecha_default = fechas_disponibles[-1]

fechas_str = [str(f) for f in fechas_disponibles]
fecha_default_str = str(fecha_default)

# 🗺️ Función para crear el mapa
def crear_mapa_interactivo(variable, fecha_obj):
    df_dia = df[df["valid_time"].dt.date == fecha_obj].copy()

    if df_dia.empty:
        return f"No hay datos para la fecha {fecha_obj}"

    # ✅ Usar colormap que sí esté disponible
    colormap = linear.YlGnBu_09.scale(df_dia[variable].min(), df_dia[variable].max())
    colormap.caption = nombre_variables_legibles[variable]

    lat_caldas = df_dia["latitude"].mean()
    lon_caldas = df_dia["longitude"].mean()
    m = folium.Map(location=[lat_caldas, lon_caldas], zoom_start=8, tiles="CartoDB positron")

    for _, row in df_dia.iterrows():
        valor = row[variable]
        popup_text = f"{nombre_variables_legibles[variable]}: {valor:.2f}"
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,
            color=colormap(valor),
            fill=True,
            fill_color=colormap(valor),
            fill_opacity=0.7,
            tooltip=popup_text
        ).add_to(m)

    m.add_child(colormap)
    return m

# 🎛️ Widget interactivo
@interact(
    variable=Dropdown(
        options=[(nombre_variables_legibles[v], v) for v in nombres_variables],
        value=nombres_variables[0],
        description="Variable:"
    ),
    fecha=Dropdown(
        options=fechas_str,
        value=fecha_default_str,
        description="Fecha:"
    )
)
def mostrar_mapa(variable, fecha):
    fecha_obj = pd.to_datetime(fecha).date()
    display(crear_mapa_interactivo(variable, fecha_obj))


interactive(children=(Dropdown(description='Variable:', options=(('Temperatura del aire (°C)', 'TA_1_1_1'), ('…